In [1]:
# Imports
import numpy as np
import spikeinterface as si

In [2]:
# Properties of the in-memory dataset
num_channels=7
samplerate=30000
duration=20
num_timepoints=int(samplerate*duration)
num_units=4
num_events=1000

In [16]:
# Generate a pure-noise timeseries dataset and a linear geometry
timeseries=np.random.normal(0,10,(num_channels,num_timepoints))
geom=np.zeros((num_channels,2))
geom[:,0]=range(num_channels)

# Define the in-memory recording extractor
RX=si.NumpyRecordingExtractor(timeseries=timeseries,geom=geom,samplerate=samplerate)

# Generate some random events
times=np.int_(np.sort(np.random.uniform(0,num_timepoints,num_events)))
labels=np.random.randint(1,num_units+1,size=num_events)
    
# Define the in-memory sorting extractor
SX=si.NumpySortingExtractor()
for k in range(1,num_units+1):
    times_k=times[np.where(labels==k)[0]]
    SX.addUnit(unit_id=k,times=times_k)

In [27]:
# Demonstrate the API for extracting information
print('Unit ids = {}'.format(SX.getUnitIds()))
st=SX.getUnitSpikeTrain(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=SX.getUnitSpikeTrain(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1, 2, 3, 4]
Num. events for unit 1 = 253
Num. events for first second of unit 1 = 14


array([ 1396,  1727,  1785,  4917,  8282, 11792, 13455, 15190, 18960,
       21855, 22685, 24976, 28039, 29052])

In [18]:
# Write the input/output in the MountainSort format
si.MdaRecordingExtractor.writeRecording(recording_extractor=RX,save_path='sample_mountainsort_dataset')
si.MdaSortingExtractor.writeSorting(sorting_extractor=SX,save_path='sample_mountainsort_dataset/firings_true.mda')

In [19]:
# Read this dataset with the Mda input extractor
RX2=si.MdaRecordingExtractor(dataset_directory='sample_mountainsort_dataset')
SX2=si.MdaSortingExtractor(firings_file='sample_mountainsort_dataset/firings_true.mda')

In [26]:
# We should get he same information as above
print('Unit ids = {}'.format(SX2.getUnitIds()))
st=SX2.getUnitSpikeTrain(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=SX2.getUnitSpikeTrain(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1 2 3 4]
Num. events for unit 1 = 253
Num. events for first second of unit 1 = 14


array([ 1396.,  1727.,  1785.,  4917.,  8282., 11792., 13455., 15190.,
       18960., 21855., 22685., 24976., 28039., 29052.])

In [32]:
#Calculate a unit property and store it in my SortingExtractor
full_spike_train = SX2.getUnitSpikeTrain(unit_id=1,start_frame=0)
print(full_spike_train)

[  1396.   1727.   1785.   4917.   8282.  11792.  13455.  15190.  18960.
  21855.  22685.  24976.  28039.  29052.  34105.  37595.  39440.  40976.
  41969.  42388.  46291.  48909.  50595.  52553.  54204.  57644.  60252.
  61924.  62896.  64012.  65906.  68708.  68797.  69190.  72001.  73622.
  75325.  75722.  75819.  78342.  78414.  78448.  79108.  80029.  82388.
  84180.  85364.  86391.  86506.  87078.  88909.  89449.  90054.  92040.
  93696.  98511.  99425. 107916. 107951. 112079. 115130. 119020. 119217.
 119850. 120209. 123739. 124544. 126430. 127007. 127473. 128283. 129353.
 130975. 131454. 134865. 139799. 141141. 142918. 151693. 153436. 154353.
 155917. 156270. 157387. 158391. 159672. 161852. 164622. 167586. 173661.
 173954. 175345. 181142. 184627. 191652. 195621. 196943. 196974. 201291.
 201533. 202434. 202555. 205565. 209730. 210581. 210646. 211169. 223766.
 227829. 228968. 229131. 232805. 234704. 234717. 237936. 238813. 240868.
 252279. 254272. 254325. 256467. 257002. 257147. 26